# Assignment 3
## initialize

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, countDistinct, avg, min, max, expr, explode, round, date_format, concat, lit

spark = SparkSession.builder.appName("ParkingDataAnalysis").getOrCreate()
data = spark.read.csv('data/parking_data_sz.csv', header=True, inferSchema=True)
# print(data.columns)  # ['out_time', 'admin_region', 'in_time', 'berthage', 'section']
data = data.filter(col("out_time") > col("in_time"))
data = data.withColumn("berthage", col("berthage").cast("string"))
# data.show()

## task1

In [2]:
result1 = data.groupBy("section").agg(
    countDistinct("berthage").alias("count")
)
# result1.show()
# result1_pd = result1.toPandas()
# result1_pd.to_csv("results/r1.csv", index=False, header=True)

## task2

In [3]:
result2 = data.select("berthage", "section").distinct()
# result2.show()
# result2_pd = result2.toPandas()
# result2_pd.to_csv("results/r2.csv", index=False, header=True)

## task3

In [4]:
result3 = data.withColumn("parking_time", (col("out_time") - col("in_time")))
result3 = result3.groupBy("section").agg(
    avg("parking_time").cast("int").alias("avg_parking_time")
)
# result3.show()
# result3_pd = result3.toPandas()
# result3_pd.to_csv("results/r3.csv", index=False, header=True)

## task4

In [5]:
result4 = data.withColumn("parking_time", (col("out_time") - col("in_time")))
result4 = result4.groupBy("berthage").agg(
    avg("parking_time").cast("int").alias("avg_parking_time")
)
# result4.show()
# result4_pd = result4.toPandas()
# result4_pd.to_csv("results/r4.csv", index=False, header=True)

## task5

In [2]:
# 5.1 set time limitation
time_limitation = data.groupBy("section").agg(
    min("in_time").alias("min_in"),
    max("out_time").alias("max_out")
)
time_limitation = time_limitation \
    .withColumn("hour_range", expr("sequence(min_in, max_out, interval 1 hour)").cast("array<timestamp>")) \
    .withColumn("start_time", explode("hour_range")) \
    .withColumn("end_time", expr("start_time + INTERVAL 1 HOUR"))
time_limitation = time_limitation \
    .select("section", "start_time", "end_time") \
    .orderBy("section", "start_time")
# time_limitation.show()
# 5.2 prepare all data needed
all_data_needed = time_limitation.alias("time_limitation").join(
    data.alias("data"),
    (data.section == time_limitation.section) &
    (data.in_time < time_limitation.end_time) &
    (data.out_time > time_limitation.start_time),
    "left"
)
all_data_needed = all_data_needed \
    .select("time_limitation.section", "start_time", "end_time", "berthage") \
    .orderBy("section", "start_time")
# all_data_needed.show()
### 5.3 calculate count & percentage
in_use = all_data_needed.groupBy("section", "start_time", "end_time").agg(
    countDistinct("berthage").cast("long").alias("count")
)
total_count = all_data_needed.groupBy("section").agg(
    countDistinct("berthage").cast("long").alias("total_count")
)
result5 = in_use \
    .join(total_count, "section") \
    .withColumn("percentage", round(col("count") / col("total_count") * 100, 1).cast("string")) \
    .select("section", "start_time", "end_time", "count", "percentage") \
    .orderBy("section", "start_time") \
    .withColumn("start_time", date_format("start_time", "yyyy-MM-dd HH:mm:ss")) \
    .withColumn("end_time", date_format("end_time", "yyyy-MM-dd HH:mm:ss")) \
    .withColumn("percentage", concat(col("percentage"), lit("%")))
# result5.show()
# result5_pd = result5.toPandas()
# result5_pd.to_csv("results/r5.csv", index=False, header=True)

### subtask

In [3]:
import plotly.express as px
import pandas as pd

sections = result5.select("section").distinct().limit(3).collect()
# print(sections)

for section in sections:
    section_name = section.section
    section_data = result5.filter(col("section") == section_name)
    x_array = [node[0] for node in section_data.select("start_time").collect()]
    y_array = [float(node[0].split("%")[0]) for node in section_data.select("percentage").collect()]
    data = pd.DataFrame({'Time': x_array, 'Percentage': y_array})
    fig = px.line(data, x='Time', y='Percentage', title=f"Percentage of Berthages in {section_name}")
    fig.show()
    break
    # fig.write_html(f'results/task5/figure_of_{section_name}.html')